In [10]:
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import pickle
import IPython.display as ipd

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [2]:
ref = pd.read_csv("Data_path.csv")
ref.head()

,labels,source,path
0,male_angry,SAVEE,SAVEE/ALL/DC_a01.wav
1,male_angry,SAVEE,SAVEE/ALL/DC_a02.wav
2,male_angry,SAVEE,SAVEE/ALL/DC_a03.wav
3,male_angry,SAVEE,SAVEE/ALL/DC_a04.wav
4,male_angry,SAVEE,SAVEE/ALL/DC_a05.wav


In [3]:
df = pd.DataFrame(columns=['feature'])

counter=0
for index,path in enumerate(ref.path):
    X, sample_rate = librosa.load(path, res_type='kaiser_fast', duration=2.5, sr=44100, offset=0.5)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
    df.loc[counter] = [mfccs]
    counter=counter+1   

print(len(df))
df.head()

12162


,feature
0,"[-4.6414213, -3.8608985, -6.21919, -5.9265423,..."
1,"[-8.690717, -12.522839, -22.928043, -23.243807..."
2,"[-8.81486, -12.819055, -24.178183, -23.84745, ..."
3,"[-2.2684255, -4.3170767, -12.285238, -13.08302..."
4,"[-13.485307, -16.26042, -25.884357, -27.827045..."


In [4]:
df = pd.concat([ref,pd.DataFrame(df['feature'].values.tolist())],axis=1)
df.to_csv('extracted.csv',index=False)
df[:5]

,labels,source,path,0,1,2,3,4,5,6,...,206,207,208,209,210,211,212,213,214,215
0,male_angry,SAVEE,SAVEE/ALL/DC_a01.wav,-4.641421,-3.860898,-6.219190,-5.926542,-5.850419,-4.808960,-2.513003,...,-4.088851,-5.023864,-5.254714,-5.234093,-5.310306,-5.621665,-6.072197,-6.611348,-3.999875,1.390507
1,male_angry,SAVEE,SAVEE/ALL/DC_a02.wav,-8.690717,-12.522839,-22.928043,-23.243807,-22.926605,-23.432243,-14.830002,...,-22.627256,-22.633406,-22.511599,-24.300154,-24.496809,-22.895985,-23.511505,-24.342152,-16.465857,-8.936036
2,male_angry,SAVEE,SAVEE/ALL/DC_a03.wav,-8.814860,-12.819055,-24.178183,-23.847450,-15.182783,-10.732485,-8.681472,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,male_angry,SAVEE,SAVEE/ALL/DC_a04.wav,-2.268425,-4.317077,-12.285238,-13.083024,-12.041327,-11.819768,-9.414147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,male_angry,SAVEE,SAVEE/ALL/DC_a05.wav,-13.485307,-16.260420,-25.884357,-27.827045,-27.593534,-26.666512,-18.659025,...,-25.291666,-25.854906,-26.821354,-25.436455,-24.179941,-23.281618,-24.167494,-25.228062,-20.599659,-15.929615


In [6]:
df.isna().sum()

labels       0
source       0
path         0
0            0
1            0
          ... 
211       8979
212       8980
213       9091
214       9093
215       9095
Length: 219, dtype: int64

In [7]:
df=df.fillna(0)
df.isna().sum()

labels    0
source    0
path      0
0         0
1         0
         ..
211       0
212       0
213       0
214       0
215       0
Length: 219, dtype: int64

In [8]:
df.shape

(12162, 219)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['path','labels','source'],axis=1), df.labels, test_size=0.2, shuffle=True, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((9729, 216), (2433, 216), (9729,), (2433,))

In [17]:
# normalizing the data
mean = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)
x_train = (x_train - mean)/std
x_test = (x_test - mean)/std

In [18]:
# pca = PCA(n_components=0.95)
# pca.fit(x_train)
# x_train = pca.transform(x_train)
# x_test = pca.transform(x_test)

In [19]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [20]:
# label encoding
lb = LabelEncoder()
y_train = tf.keras.utils.to_categorical(lb.fit_transform(y_train))
y_test = tf.keras.utils.to_categorical(lb.fit_transform(y_test))

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((9729, 216), (2433, 216), (9729, 14), (2433, 14))

In [21]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, x_test.shape

((9729, 216, 1), (2433, 216, 1))

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(256, 8, padding='same', activation='relu', input_shape=(x_train.shape[1],1)),
    tf.keras.layers.Conv1D(256, 8, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.MaxPooling1D(pool_size=(8)),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(128, 8, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.MaxPooling1D(pool_size=(8)),
    tf.keras.layers.Conv1D(64, 8, padding='same', activation='relu'),
    tf.keras.layers.Conv1D(64, 8, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(14, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 216, 256)          2304      
                                                                 
 conv1d_9 (Conv1D)           (None, 216, 256)          524544    
                                                                 
 batch_normalization_2 (Batc  (None, 216, 256)         1024      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 216, 256)          0         
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 27, 256)          0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 27, 128)          

c:\Users\Fadil\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test), verbose=1)

Epoch 1/100
305/305 [==============================] - 5s 15ms/step - loss: 2.1892 - accuracy: 0.2449 - val_loss: 2.5018 - val_accuracy: 0.1517
Epoch 2/100
305/305 [==============================] - 4s 15ms/step - loss: 1.9135 - accuracy: 0.3237 - val_loss: 2.3614 - val_accuracy: 0.2462
Epoch 3/100
305/305 [==============================] - 4s 14ms/step - loss: 1.8047 - accuracy: 0.3632 - val_loss: 2.1692 - val_accuracy: 0.3033
Epoch 4/100
305/305 [==============================] - 4s 13ms/step - loss: 1.7552 - accuracy: 0.3838 - val_loss: 2.1405 - val_accuracy: 0.3181
Epoch 5/100
305/305 [==============================] - 4s 13ms/step - loss: 1.7016 - accuracy: 0.4005 - val_loss: 2.1205 - val_accuracy: 0.3103
Epoch 6/100
305/305 [==============================] - 4s 13ms/step - loss: 1.6559 - accuracy: 0.4175 - val_loss: 1.9920 - val_accuracy: 0.3457
Epoch 7/100
305/305 [==============================] - 4s 14ms/step - loss: 1.6159 - accuracy: 0.4298 - val_loss: 2.1557 - val_accuracy:

In [31]:
model.save("model1.h5")